### Fitting coasting cosmological model to the QSO database

- The aim of this code is to constrain coasting cosmological models by fitting them to the QSO database. The fitting is performed using an MCMC algorithm that minimizes the $\chi^2$ statistic based on the distance moduli calculated from the model and observed data. <br>
The fitting process consists of two steps: <br>
Step #1 - Finding outliers: With **SigmaClipping** set to True, outlier QSOs are identified through automatic sigma clipping over multiple iterations. <br>
Step #2 - Constraining parameters: With **SigmaClipping** set to False, QSO and model parameters are constrained using only the non-discarded QSOs. <br>
<br>
<ins>Parameters:</ins> <br>
- **SigmaClipping**: True or False. <br>
$\quad$ $\quad$ True - Finding outliers, <br>
$\quad$ $\quad$ False - Constraining QSO and model parameters. <br>
- **ParamNames**: List of fitted parameters. The free parameters of QSOs ($\gamma$, $\beta$) and the model ($h = H_0/100,\textrm{km},\textrm{s}^{-1},\textrm{Mpc}^{-1}$, the reduced Hubble constant) are fitted simultaneously. <br>
- **Labels**: Formatted parameter names as they appear on the corner plot. <br>
- **PriorInd**: Indices of parameters with non-uniform priors. Gaussian prior mean and sigma are specified in **parameter_priors_*Prefix*.txt**. Parameters not listed in **PriorInd** use uniform priors within the ranges defined in **parameter_limits.txt**, disregarding **parameter_priors_*Prefix*.txt**. The following setting was applied: <br>
$\quad$ $\quad$ PriorInd = [2] - $H_0$ prior. <br>
- **k**: Curvature parameter of the coasting model, in $H_0^2\,c^{-2}$ units, set to -1, 0 or 1. <br>
- **TrialNum**: Number of MCMC steps (integer). Based on preliminary runs, **TrialNum** is set to: <br>
$\quad$ $\quad$ 5,000,000 if **SigmaClipping** = True, <br>
$\quad$ $\quad$ 50,000,000 if **SigmaClipping** = False. <br>
- **printstep**: Displays the number of steps and elapsed time in seconds after every **printstep** steps in the MCMC process. **printstep** must be an integer. <br>
- **burn**: Burn-in period (integer). The first **burn** steps are discarded to determine optimal parameters. Based on preliminary runs, **burn** is set to 100,000 steps for all coasting models. <br>
- **bins**: Number of bins for the histograms on the corner plot. <br>
<br>
<ins>Calculated parameters:</ins> <br>
- **NumParams**: Number of fitted parameters, equal to the length of **ParamNames**. <br>
- **Prefix**: String used to distinguish different fitted models, based on the curvature parameter and any parameters with priors. It is appended to the generated filenames. If **SigmaClipping** is True, **Prefix** is further extended with **sc _IterationNumber_**. <br>
<br>
<ins>Input files</ins> (Must be in the same folder as this code!): <br>
- **parameter_limits.txt**: Defines parameter space boundaries (column #1: lower, column #2: upper boundary) to be explored in the fitting process, as well as the step length (column #3) applied by the MCMC algorithm along the different dimensions of the parameter space. Each row corresponds to one fitted parameter, in the same order defined in **ParamNames**. Data from txt is loaded into **parameter_limits**. <br>
- **parameter_priors_*Prefix*.txt**: Specifies Gaussian prior mean (column #1) and sigma (column #2: left, column #3: right uncertainty) for fitted parameters. Each row corresponds to a parameter, following the order in **ParamNames**. Only rows with indices in **PriorInd** are used. Data from txt is loaded into **parameter_priors**, where the average of left and right uncertainties is calculated. This mean uncertainty is used during the **MCMC** algorithm. Prior was applied for one parameter: <br>
$\quad$ $\quad$ $H_0$ prior: Value and uncertainty for $H_0$, obtained from fitting the corresponding coasting model to the SNIa database with $M_B+H_0$ prior. <br>
- **QSO_data.txt**: QSO database, available in 'table3.dat' at http://cdsarc.u-strasbg.fr/viz-bin/cat/J/A+A/642/A150. Data is stored in **QSO_data**, where the entire database is restricted to the following columns: <br>
$\quad$ $\quad$ #4: z - redshift, <br>
$\quad$ $\quad$ #5: logFUV - rest-frame flux at 2500 Angstroem in units of log(erg/s/cm^2), <br>
$\quad$ $\quad$ #6: e_logFUV -  error on rest-frame flux at 2500 Angstroem, <br>
$\quad$ $\quad$ #7: logFX - rest-frame flux at 2keV in units of log(erg/s/cm^2), <br>
$\quad$ $\quad$ #8: e_logFX - error on rest-frame flux at 2keV. <br>
<br>
<ins>Functions:</ins> <br>
- **Mu_measured**: Returns the distance moduli (array **mu_meas**) and its sigma-squared (array **mu_meas_ssq**) for each QSO, calculated from the measurement data using the following parameters: <br>
$\quad$ $\quad$ logFUV, logFUV_s, logFX, logFX_s: measured data, comes from the array **QSO_data**, <br>
$\quad$ $\quad$ r_logFUV_logFX: Pearson correlation coefficient between logFUV and logFX, that is used in calculating the covariant term in **mu_meas_ssq**, <br>
$\quad$ $\quad$ gamma, beta: free QSO parameters. <br>
- **Mu_theory**: Returns the distance moduli for each QSO (array **mu_th**) calculated from the cosmological model using the following parameters: <br>
$\quad$ $\quad$ z : redshift - measured data, comes from the array **QSO_data**, <br>
$\quad$ $\quad$ h : reduced Hubble constant - free parameter of the model, <br>
$\quad$ $\quad$ k : curvature parameter - must be set in Parameters, <br>
$\quad$ $\quad$ c : speed of light in vacuum expressed in m/s units - hardcoded parameter. <br>
- **ChiCalculator**: Calculates and returns $\chi^2$ for each QSO (array **Chi**) from **mu_meas**, **mu_meas_ssq** and **mu_th**. <br>
<br>
<ins>MCMC algorithm:</ins> <br>
- In the **MCMC** function we start from a randomly chosen point in parameter space, with coordinates drawn uniformly between parameter boundaries, and compute the initial $\chi^2$ statistic (**OutStat**). <br>
We then take **TrialNum** steps, each time calculating the $\chi^2$ statistic (**TestOutStat**) with updated parameters (**TestParams**). Each step involves randomly selecting a parameter (**ParamInd**) and choosing a step length from a Gaussian distribution with zero mean and sigma defined in **parameter_limits** (**RandStepParam**). If a step moves out of bounds, we adjust in the opposite direction. <br>
Steps are accepted with probability 1 if $\chi^2$ improves, or with probability **a** if it worsens. Otherwise the new step is rejected and we keep the previous step's parameters. The acceptance probability **a** includes a factor **b**, based on prior knowledge of the parameter we stepped with, otherwise **b** is 1. <br>
Each step is recorded in the file **OutMatrix_*Prefix*.txt**. <br>
<br>
<ins>Optimal parameter values:</ins> <br>
- **OptPar** function determines optimal parameters with left and right uncertainties as the 0.5, 0.16 and 0.84 quantiles. It processes values from **OutMatrix_*Prefix*.txt**, excluding the first **burn** steps. The result is written in a file named **params_*Prefix*.txt**. <br>
<br>
<ins>Sigma clipping:</ins> <br>
- If **SigmaClipping** is True, sigma clipping is applied automatically (function **Sigma_Clipping**), with multiple iterations per fitting process. After each iteration, QSOs with distance moduli deviating more than 3 sigma from the model, based on the current parameters, are filtered out. <br>
<br>
<ins>RunThis:</ins> <br>
- If **SigmaClipping** is True, the function **RunThis** executes the entire iteration process by calling **MCMC**, **OptPar** and **Sigma_Clipping**. The string **sc** along with the current iteration number (**IterationNumber**) is appended to filenames. An automatically generated file **outfiltered_*Prefix*.txt** records the iteration number at which each QSO was discarded (or 0 if not rejected). The start and end of each iteration, along with the count of outfiltered QSOs, are displayed on screen. The process continues until no more outliers are found. <br>
- If **SigmaClipping** is False, the function **RunThis** calls **MCMC** and **OptPar** using the non-discarded QSOs only. <br>
<br>
<ins>Output files</ins> (Generated in the same location as the code.): <br>
- **OutMatrix_*Prefix*.txt**: MCMC output containing parameter values and $\chi^2$ statistics. It has # parameters + 1 columns and TrialNum + 1 rows (including the initial configuration), with a new line added after each step. During Step #1 when **SigmaClipping** is True, a new file is created for each iteration, distinguished by appending **sc _IterationNumber_** to the filename. In Step #2 one such file is created. <br>
- **outfiltered_*Prefix*.txt**: Matches the QSO database in rows and records the IterationNumber when each QSO was rejected. Only one such file is created during Step #1 when **SigmaClipping** is True, and this file is used in Step #2. <br>
- **params_*Prefix*.txt**: Contains parameter values (column #1) and uncertainties (columns #2: left, #3: right), with rows corresponding to parameters in the order defined by **ParamNames**. During Step #1 when **SigmaClipping** is True, a new file is created for each iteration, distinguished by appending **sc _IterationNumber_** to the filename. In Step #2 one such file is created. <br>
- **opt_params_*Prefix*.png**: Corner plot of the fitted parameters, generated at the conclusion of Step #2.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import time
import corner

#### Parameters

In [3]:
SigmaClipping = True
ParamNames    = ['gamma','beta','h']
Labels        = [r'$\gamma$',r'$\beta$',r'$h$']
PriorInd      = [2]
k             = 0
TrialNum      = int((5 if SigmaClipping else 50) * 1e6)
printstep     = int(TrialNum/10)
burn          = 100000
bins          = 100
#############################################################################################
NumParams     = len(ParamNames)
Prefix        = '_k' + str(int(k)) + '_' + ''.join([ParamNames[i] for i in PriorInd]) + '-SNMBhprior'
print(Prefix)

_k0_h-SNMBhprior


#### Input files

In [4]:
parameter_limits  = np.genfromtxt('parameter_limits.txt', delimiter='\t')
parameter_priors1 = np.genfromtxt('parameter_priors' + Prefix + '.txt', delimiter='\t')
sigma_mean        = [np.mean([parameter_priors1[i][1],parameter_priors1[i][2]]) for i in range(len(parameter_priors1))]
parameter_priors  = np.insert(parameter_priors1, 1, sigma_mean, axis=1)
QSO_data_orig     = np.genfromtxt('QSO_data.txt', delimiter='\t')
QSO_data          = QSO_data_orig[QSO_data_orig[:,0].argsort(kind='mergesort')]
del parameter_priors1, QSO_data_orig

#### Functions

In [5]:
c = 299792458
r_logFUV_logFX = np.corrcoef(QSO_data[:,1], QSO_data[:,3])[0][1]

def Mu_measured(QSO_dat, Params):    
    [z, logFUV, logFUV_s, logFX, logFX_s] = [QSO_dat[:,i] for i in range(len(QSO_dat[0]))]     
    [gamma, beta, h]                      = [Params[i] for i in range(NumParams)]  
    mu_meas     = (5/(2*(gamma-1)))*(logFX-gamma*logFUV) - beta    
    mu_meas_ssq = ((5/(2*(gamma-1)))**2) * (logFX_s**2 + gamma**2 * logFUV_s**2 - \
                  2*gamma * r_logFUV_logFX * logFX_s * logFUV_s)   
    return mu_meas, mu_meas_ssq

def Mu_theory(QSO_dat, Params):    
    [z, logFUV, logFUV_s, logFX, logFX_s] = [QSO_dat[:,i] for i in range(len(QSO_dat[0]))]     
    [gamma, beta, h]                      = [Params[i] for i in range(NumParams)]    
    if k == 0:
        mu_th = 5 * np.log10(c*(1+z) * np.log(1+z)) - 5 * np.log10(h)
    elif k == 1:
        mu_th = 5 * np.log10(c*(1+z) * np.sin(np.log(1+z))) - 5 * np.log10(h)
    elif k == -1:
        mu_th = 5 * np.log10(c*(1+z) * np.sinh(np.log(1+z))) - 5 * np.log10(h)
    return mu_th

def ChiCalculator(QSO_dat, Params):    
    mu_meas, mu_meas_ssq = Mu_measured(QSO_dat, Params)
    mu_th                = Mu_theory(QSO_dat, Params)    
    Chi                  = ((mu_meas - mu_th)**2)/mu_meas_ssq    
    return Chi

#### MCMC algorithm

In [ ]:
def MCMC(QSO_dat, Prefix):
    
    # Initial configuration
    Params        = np.empty(NumParams)
    for i in range(NumParams):
        Params[i] = parameter_limits[i][0] + (parameter_limits[i][1] - parameter_limits[i][0]) * random.random()
    OutStat       = sum(ChiCalculator(QSO_dat, Params))
    Params_out    = np.append(Params, OutStat)
    np.savetxt('OutMatrix' + Prefix + '.txt', Params_out, newline=' ') 
        
    # Wandering within the the parameter space for TrialNum steps
    for j in range(TrialNum):
        if j/printstep == np.floor(j/printstep):
            start = time.time()
        
        Test_Params   = Params.copy()
        ParamInd      = random.randint(1, NumParams)-1        
        RandStepParam = parameter_limits[ParamInd][2] * random.gauss(0,1)
        
        if parameter_limits[ParamInd][0] <= Test_Params[ParamInd]+RandStepParam <= parameter_limits[ParamInd][1]:
            Test_Params[ParamInd]   = Test_Params[ParamInd] + RandStepParam
        else: Test_Params[ParamInd] = Test_Params[ParamInd] - RandStepParam            
        
        Test_OutStat = sum(ChiCalculator(QSO_dat, Test_Params))

        if ParamInd in PriorInd:
            b = np.exp(((Params[ParamInd]      - parameter_priors[ParamInd][0])**2 - \
                        (Test_Params[ParamInd] - parameter_priors[ParamInd][0])**2) / \
                        (2 * parameter_priors[ParamInd][1]**2))
        else: b = 1
   
        a = np.exp(-(Test_OutStat-OutStat)/2) * b
        p = random.random()
        if a >= p:
            OutStat = Test_OutStat
            Params  = Test_Params

        Params_out   = np.append(Params, OutStat)
        with open('OutMatrix' + Prefix + '.txt','a') as f:
            f.write('\n')
            np.savetxt(f, Params_out, newline=' ')
                
        if (j+1)/printstep == np.floor((j+1)/printstep):
            end = time.time()
            print(j+1, end-start)

#### Optimal parameter values

In [ ]:
def OptPar(Prefix):
    
    OutMatrix = np.genfromtxt('OutMatrix' + Prefix + '.txt', delimiter=' ')
    OutM      = OutMatrix[:,range(NumParams)][burn:]    
    quantiles = [0.16, 0.5, 0.84]
    
    OptParams = []
    for i in range(NumParams):
        opt       = corner.quantile(OutM[:,i],quantiles)
        OptParams = OptParams + [[opt[1], opt[1]-opt[0], opt[2]-opt[1]]]
    np.savetxt('params' + Prefix + '.txt', OptParams, delimiter='\t')
    
    if SigmaClipping == False:
        figure = corner.corner(OutM, bins=bins, labels=Labels, quantiles=quantiles, show_titles=True, title_fmt='.4f', title_kwargs={"fontsize": 12})
        figure.savefig('opt_params' + Prefix + '.png')

#### Sigma clipping

In [ ]:
def Sigma_Clipping(Prefix, Prefix2, IterationNumber):
    
    Params        = np.genfromtxt('params' + Prefix2 + '.txt')[:,0]
    Chi           = ChiCalculator(QSO_data, Params)
    outf          = np.sqrt(Chi) > 3
    
    Outfiltered   = np.genfromtxt('outfiltered' + Prefix + '.txt')
    for i in range(len(Outfiltered)):
        if (Outfiltered[i] == 0 and outf[i] == True):
            Outfiltered[i] = IterationNumber    
    np.savetxt('outfiltered' + Prefix + '.txt', Outfiltered)

#### RunThis

In [ ]:
def RunThis():
    
    if SigmaClipping:    
        IterationNumber = 0
        OutfiltCount    = 1
    
        while OutfiltCount > 0:
        
            IterationNumber += 1
            print('Start iteration #' + str(IterationNumber))
            
            Prefix2 = Prefix + '_sc' + str(IterationNumber)
        
            if IterationNumber == 1:
                np.savetxt('outfiltered' + Prefix + '.txt', [0 for i in range(len(QSO_data))])
                Data         = QSO_data
            else:
                Outfiltered  = np.genfromtxt('outfiltered' + Prefix + '.txt')
                Data         = QSO_data[Outfiltered == 0]
              
            MCMC(Data, Prefix2)
            OptPar(Prefix2)
            Sigma_Clipping(Prefix, Prefix2, IterationNumber)
        
            Outfiltered  = np.genfromtxt('outfiltered' + Prefix + '.txt')
            OutfiltCount = sum(Outfiltered == IterationNumber)
        
            print('End iteration #' + str(IterationNumber) + '; Outfiltered: ' + str(OutfiltCount))
        
    else:
        Outfiltered  = np.genfromtxt('outfiltered' + Prefix + '.txt')
        Data         = QSO_data[Outfiltered == 0]
        
        MCMC(Data, Prefix)
        OptPar(Prefix)

In [ ]:
RunThis()